In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install transformers[torch] datasets accelerate evaluate -q

print("Libraries installed and Google Drive mounted.")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
✅ Libraries installed and Google Drive mounted.


In [ ]:
import transformers
print(transformers.__version__)


4.57.1


In [ ]:
# Cell 2: Load Dataset and Tokenizer

from datasets import load_dataset
from transformers import AutoTokenizer

# 1. Load the Emotion dataset
print("Loading the 'dair-ai/emotion' dataset...")
emotion_dataset = load_dataset("dair-ai/emotion")

# 2. Load the tokenizer for DeBERTa V3 Base
model_checkpoint = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# 3. Create a function to tokenize the text
def tokenize_function(examples):
    # The tokenizer will convert text to the numbers (input_ids) the model expects.
    # We use truncation to handle texts longer than the model's max input size.
    return tokenizer(examples["text"], truncation=True)

# 4. Apply the tokenizer to the entire dataset
print("Tokenizing dataset (this may take a minute)...")
tokenized_datasets = emotion_dataset.map(tokenize_function, batched=True)

# We can remove the original 'text' column as it's no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
# Rename the 'label' column to 'labels' which the model expects
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# Set the format to PyTorch tensors
tokenized_datasets.set_format("torch")

print("\nDataset is ready for training.")
# Note: The dataset is split into train, validation, and test. We'll use them accordingly.
print(f"Sample of processed data: {tokenized_datasets['train'][0]}")

Loading the 'dair-ai/emotion' dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
 

Tokenizing dataset (this may take a minute)...


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Dataset is ready for training.
Sample of processed data: {'labels': tensor(0), 'input_ids': tensor([    1,   584, 16965,   551, 41064,     2]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1])}


In [ ]:
import numpy as np
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# 1. Define where to save the model on your Google Drive
output_model_dir = "/content/drive/MyDrive/deberta-v3-base-emotion-finetuned"

# 2. Create label mappings for clarity
labels = emotion_dataset["train"].features["label"].names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# 3. Load the pre-trained model
# Set num_labels=6 for the six emotions
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=6,
    id2label=id2label,
    label2id=label2id
)

# 4. Define a function to compute metrics (we'll use accuracy)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 5. Define Training Arguments (CORRECTED for transformers 4.5+)
training_args = TrainingArguments(
    output_dir="temp_emotion_model_output",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    logging_steps=100,
)

# 6. Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 7. Start the training!
print("Starting fine-tuning for emotion classification...")
trainer.train()
print("Training complete.")

# 8. Save the final, best-performing model to your Google Drive
print(f"Saving final model to: {output_model_dir}")
trainer.save_model(output_model_dir)
tokenizer.save_pretrained(output_model_dir)
print("All done! Your fine-tuned emotion classification model is saved in Google Drive.")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-229838942.py:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


🚀 Starting fine-tuning for emotion classification...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ragunathravi73 (ragunathravi73-saveetha-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.241200,0.249670,0.925000
2,0.127800,0.176021,0.937500


✅ Training complete.
💾 Saving final model to: /content/drive/MyDrive/deberta-v3-base-emotion-finetuned
All done! Your fine-tuned emotion classification model is saved in Google Drive.


# **Test**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_dir = "/content/drive/MyDrive/deberta-v3-base-emotion-finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

print("Model and tokenizer loaded from:", model_dir)


Mounted at /content/drive
✅ Model and tokenizer loaded from: /content/drive/MyDrive/deberta-v3-base-emotion-finetuned


In [ ]:
import torch.nn.functional as F

def classify_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1).cpu().numpy()[0]

    id2label = {int(k): v for k, v in model.config.id2label.items()}
    top_idx = int(probs.argmax())
    predicted_label = id2label[top_idx]
    confidence = probs[top_idx]

    print(f"\nText: {text}")
    print(f"Predicted Emotion: {predicted_label}  (Confidence: {confidence:.4f})")
    print("\n--- Confidence for all classes ---")
    for i, p in enumerate(probs):
        print(f"{id2label[i]:<10}: {p:.4f}")

# Example prediction
classify_text("I am so happy today!")

# User input
user_input = input("\nEnter text to classify: ")
if user_input.strip():
    classify_text(user_input)
else:
    print("No input provided.")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Text: I am so happy today!
Predicted Emotion: joy  (Confidence: 0.9995)

--- Confidence for all classes ---
sadness   : 0.0000
joy       : 0.9995
love      : 0.0003
anger     : 0.0000
fear      : 0.0000
surprise  : 0.0001

Enter text to classify: i saw a poor man today i think he must have some good life 

Text: i saw a poor man today i think he must have some good life 
Predicted Emotion: sadness  (Confidence: 0.4734)

--- Confidence for all classes ---
sadness   : 0.4734
joy       : 0.1514
love      : 0.0104
anger     : 0.1925
fear      : 0.1569
surprise  : 0.0153


# **Upload to huggingface hub**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers[torch] huggingface_hub accelerate

from getpass import getpass
from huggingface_hub import HfApi, login
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os
HF_TOKEN = getpass("Enter your Hugging Face write access token: ")
login(token=HF_TOKEN)
repo_id = "ragunath-ravi/deberta-v3-emotion-classifier"
model_dir = "/content/drive/MyDrive/deberta-v3-base-emotion-finetuned"
api = HfApi()
api.create_repo(repo_id=repo_id, exist_ok=True, private=False, token=HF_TOKEN)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model.push_to_hub(repo_id, use_auth_token=HF_TOKEN)
tokenizer.push_to_hub(repo_id, use_auth_token=HF_TOKEN)
readme = """# DeBERTa v3 Emotion Classifier

Repository: ragunath-ravi/deberta-v3-emotion-classifier

## Model description
This repository contains a DeBERTa v3 base model fine-tuned for emotion classification on the `dair-ai/emotion` dataset. The model is intended for short-text emotion labeling and was finetuned with standard Trainer-based training on Google Colab / Drive.

## Use cases
- Classifying short texts into emotion categories for downstream workflows (analytics, moderation, UX signals).
- Human-in-the-loop pipelines where low-confidence outputs trigger clarification.

## Dataset
- `dair-ai/emotion` (public dataset for emotion labeling).

## Training summary
- Base model: `microsoft/deberta-v3-base`
- Fine-tuning method: full fine-tuning (Trainer)
- Number of labels: 6
- Training environment: Google Colab (model checkpoint stored on Google Drive)
- Training logs and runs available at: https://wandb.ai/ragunathravi73-saveetha-engineering-college/huggingface

## Reported metrics
| Epoch | Training Loss | Validation Loss | Accuracy |
|-------|---------------:|-----------------:|---------:|
| 1     | 0.241200       | 0.249670         | 0.925000 |
| 2     | 0.127800       | 0.176021         | 0.937500 |

## Model card / intended use
This model is intended to support emotion classification tasks. It may produce incorrect or biased outputs when used on out-of-distribution text, long-form inputs, or languages it was not trained on. Use a confidence-based fallback for important decisions and include human review for high-stakes applications.

## How to load
```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
repo_id = """ + repo_id + """
tokenizer = AutoTokenizer.from_pretrained(repo_id)
model = AutoModelForSequenceClassification.from_pretrained(repo_id)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()
```

## Example inference
```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
tokenizer = AutoTokenizer.from_pretrained(""" + repo_id + """)
model = AutoModelForSequenceClassification.from_pretrained(""" + repo_id + """)
inputs = tokenizer("I am so happy today!", return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    logits = model(**inputs).logits
probs = F.softmax(logits, dim=-1).cpu().numpy()[0]
print(probs)
```

## Fallback & HIL (Human-in-the-loop) guidance
- Use prediction confidence to gate automatic decisions. If confidence < threshold (e.g., 0.7), request clarification or escalate to a human or an alternative strategy.
- Save logs of fallback activations to monitor model reliability.

## Credits & contact
Model fine-tuned by Ragunath R. Training logs available at the provided wandb link above.

## License
MIT License
"""
with open("README.md","w") as f:
    f.write(readme)
api.upload_file(path_or_fileobj="README.md", path_in_repo="README.md", repo_id=repo_id, token=HF_TOKEN)
print("Uploaded model and README to https://huggingface.co/" + repo_id)


Mounted at /content/drive
Enter your Hugging Face write access token: ··········


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:917: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/hf_api.py:9717: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


Uploaded model and README to https://huggingface.co/ragunath-ravi/deberta-v3-emotion-classifier
